In [7]:
import cv2
import os
import numpy as np

os.listdir("D:\shitcam")
vid_src = "D:\shitcam\espFeederCamera_230213204603.avi"

In [22]:
vid = cv2.VideoCapture(vid_src)
previous_frame =None
while vid.isOpened():
    ret, frame = vid.read()
    if not ret:
        print("Can,t receive frame")
        break
    
    # 2. Prepare image; grayscale and blur
    prepared_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    prepared_frame = cv2.GaussianBlur(src=prepared_frame, ksize=(5,5), sigmaX=0)
    
    # 3. Set previous frame and continue if there is None
    if (previous_frame is None):
    # First frame; there is no previous one yet
        previous_frame = prepared_frame
        continue
    
    # calculate difference and update previous frame
    diff_frame = cv2.absdiff(src1=previous_frame, src2=prepared_frame)
    previous_frame = prepared_frame

    # 4. Dilute the image a bit to make differences more seeable; more suitable for contour detection
    kernel = np.ones((5 , 5))
    diff_frame = cv2.dilate(diff_frame, kernel, 1)

    # 5. Only take different areas that are different enough (>20 / 255)
    thresh_frame = cv2.threshold(src=diff_frame, thresh=20, maxval=255, type=cv2.THRESH_BINARY)[1]
    #thresh_frame = cv2.dilate(thresh_frame, kernel, 1)
    #thresh_frame = cv2.threshold(src=thresh_frame, thresh=20, maxval=255, type=cv2.THRESH_BINARY)[1]

    contours, _ = cv2.findContours(image=thresh_frame, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
    #cv2.drawContours(image=frame, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)


    contours, _ = cv2.findContours(image=thresh_frame, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) < 50:
            # too small: skip!
            continue
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(img=frame, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=2)

    frame = cv2.hconcat([frame, cv2.cvtColor(thresh_frame,cv2.COLOR_GRAY2RGB)])

    
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()